In [1]:
from manim import *
import numpy as np
import math
config.media_width = "75%"
config.verbosity = "WARNING"         
from scipy.stats import norm
import os


Manim Community v0.17.3

In [33]:
def PDF_bivariate_normal(x_1, x_2, mu_1=0, mu_2=0, sigma_1=1, sigma_2=1, rho=0):
    '''
    General form of probability density function of bivariate normal distribution
    '''
    normalizing_const = 1/(2 * math.pi * sigma_1 * sigma_2 * math.sqrt(1 - rho**2))
    exp_coeff = -(1/(2 * (1 - rho**2)))
    A = ((x_1 - mu_1)/sigma_1)**2
    B = -2 * rho * ((x_1 - mu_1)/sigma_1) * ((x_2 - mu_2)/sigma_2)
    C = ((x_2 - mu_2)/sigma_2)**2

    return normalizing_const * math.exp(exp_coeff*(A + B + C))

In [37]:
%%manim -qm WordyScene
class WordyScene(Scene):
    def construct(self):
        equation = r"""
        \phi(x_1,x_2) = \frac{1}{2 \pi \sigma_1 \sigma_2 \sqrt{1 - \rho^2}} \exp \left( - \frac{1}{2(1 - \rho^2)} \left[ \left( \frac{x_1 - \mu_1}{\sigma_1} \right)^2 + 2 \rho \left( \frac{x_1 - \mu_1}{\sigma_1} \right) \left( \frac{x_2 - \mu_2}{\sigma_2} \right) + \left( \frac{x_2 - \mu_2}{\sigma_2} \right)^2 \right] \right)
        """

        # Create a LaTeX object from the equation
        latex_equation = MathTex(equation).to_edge(UP).scale(0.6)
        where = Tex(r'Where:').next_to(latex_equation,DOWN,buff=0.7)
        where.shift(4.5*LEFT)
        rho_formula = MathTex(r' \rho(x,y)=\frac{E[(x-\mu_x)(y-\mu_y)]}{\sigma_x\sigma_y}').next_to(where, RIGHT, buff=1)
        X = np.random.normal(loc=2.5,scale=0.8,size=150)
        Y = np.random.normal(loc=-2,scale=1.25,size=150)
        for i in range(len(X)):
            X[i] = round(X[i],2)
        for i in range(len(Y)):
            Y[i] = round(Y[i],2)
        sample = Tex(r'Sample:').next_to(where,DOWN,buff=0.7)
        table = Table(
            [[f"{X[0]}", f"{X[1]}","...",f"{X[149]}"],
            [f"{Y[0]}", f"{Y[1]}","...",f"{Y[149]}"]],
            row_labels=[Text("X"), Text("Y")],
            col_labels=[Text("1"), Text("2"), Text("..."), Text("n")],
            include_outer_lines=True).scale(0.5).next_to(sample,RIGHT+DOWN)
        muX = MathTex(r'\mu_X=2.5,').next_to(table,DOWN+LEFT)
        muX.shift(0.5*DOWN)
        sigmaX = MathTex(r'\sigma_X=0.8,').next_to(muX)
        muY = MathTex(r'\mu_Y=-2,').next_to(sigmaX)
        sigmaY = MathTex(r'\sigma_Y=1.25,').next_to(muY)
        corr = round(np.corrcoef(X,Y)[0,1],2)
        rho = MathTex(r'\rho=').next_to(sigmaY)
        cor = Tex(f'{corr}').next_to(rho,buff=0.3)
        self.play(Write(latex_equation))
        self.wait(1)
        self.play(Write(where),Write(rho_formula))
        self.wait(2)
        self.play(Write(sample),Create(table))
        self.wait(2)
        self.play(Write(muX))
        self.play(Write(sigmaX))
        self.play(Write(muY))
        self.play(Write(sigmaY))
        self.play(Write(rho))
        self.play(Write(cor))
        self.wait(2)

In [37]:


%%manim -qm MainScene1
COLOR_RAMP = [
    rgb_to_color([57/255, 0.0, 153/255]),
    rgb_to_color([158/255, 0.0, 89/255]),
    rgb_to_color([1.0, 0.0, 84/255]),
    rgb_to_color([1.0, 84/255, 0.0]),
    rgb_to_color([1.0, 189/255, 0.0])
]
class MainScene1(ThreeDScene):
    '''
    Plots the surface of the probability density function of the standard
    bivariate normal distribution
    '''
    def construct(self):
        X = np.random.normal(loc=2.5,scale=0.8,size=150)
        Y = np.random.normal(loc=-s2,scale=1.25,size=150)

        ax = ThreeDAxes(
            x_range = [-5.5, 5.5, 1],
            y_range = [-5.5, 5.5, 1],
            z_range = [0, 0.2, 0.1]
        )
        x_label = ax.get_x_axis_label(r'x_1')
        y_label = ax.get_y_axis_label(r'x_2', edge=UP, buff=0.2)
        z_label = ax.get_z_axis_label(r'\phi(x_1, x_2)', buff=0.2)
        axis_labels = VGroup(x_label, y_label, z_label)

        ##for define tracking value:
        mu_1 = ValueTracker(2.5)
        mu_2 = ValueTracker(-2)
        sigma_1 = ValueTracker(0.8)
        sigma_2 = ValueTracker(1.25)
        temp = np.corrcoef(X,Y)[0,1]
        rho = ValueTracker(temp)

        ##for text only
        mu_1_tex = MathTex(r'\mu_1 =')
        mu_1_value_text = always_redraw(
            lambda: DecimalNumber(num_decimal_places=2, include_sign=True)
            .set_value(mu_1.get_value())
            .next_to(mu_1_tex, RIGHT)
        )
        mu_1_text_group = VGroup(mu_1_tex, mu_1_value_text).arrange(RIGHT, buff=0.5)

        mu_2_tex = MathTex(r'\mu_2 =')
        mu_2_value_text = always_redraw(
            lambda: DecimalNumber(num_decimal_places=2, include_sign=True)
            .set_value(mu_2.get_value())
            .next_to(mu_2_tex, RIGHT)
        )
        mu_2_text_group = VGroup(mu_2_tex, mu_2_value_text).arrange(RIGHT, buff=0.5)

        sigma_1_tex = MathTex(r'\sigma_1 =')
        sigma_1_value_text = always_redraw(
            lambda: DecimalNumber(num_decimal_places=2, include_sign=True)
            .set_value(sigma_1.get_value())
            .next_to(sigma_1_tex, RIGHT)
        )
        sigma_1_text_group = VGroup(sigma_1_tex, sigma_1_value_text).arrange(RIGHT, buff=0.5)

        sigma_2_tex = MathTex(r'\sigma_2 =')
        sigma_2_value_text = always_redraw(
            lambda: DecimalNumber(num_decimal_places=2, include_sign=True)
            .set_value(sigma_2.get_value())
            .next_to(sigma_2_tex, RIGHT)
        )
        sigma_2_text_group = VGroup(sigma_2_tex, sigma_2_value_text).arrange(RIGHT, buff=0.5)

        rho_tex = MathTex(r'\rho =')
        rho_value_text = always_redraw(
            lambda: DecimalNumber(num_decimal_places=2, include_sign=True)
            .set_value(rho.get_value())
            .next_to(rho_tex, RIGHT)
        )
        rho_text_group = VGroup(rho_tex, rho_value_text).arrange(RIGHT, buff=0.5)

        text = VGroup(mu_1_text_group, sigma_1_text_group, mu_2_text_group, sigma_2_text_group,rho_text_group).arrange(RIGHT, buff=0.7).move_to(UP*3.5).scale(0.75)
        self.add_fixed_in_frame_mobjects(text)
        # Define the LaTeX equation
        equation = r"""
        \phi(x_1,x_2) = \frac{1}{2 \pi \sigma_1 \sigma_2 \sqrt{1 - \rho^2}} \exp \left( - \frac{1}{2(1 - \rho^2)} \left[ \left( \frac{x_1 - \mu_1}{\sigma_1} \right)^2 + 2 \rho \left( \frac{x_1 - \mu_1}{\sigma_1} \right) \left( \frac{x_2 - \mu_2}{\sigma_2} \right) + \left( \frac{x_2 - \mu_2}{\sigma_2} \right)^2 \right] \right)
        """

        # Create a LaTeX object from the equation
        latex_equation = MathTex(equation)

        # Center the LaTeX object on the screen
        latex_equation.to_edge(DOWN,buff=-0.2)
        latex_equation.scale(0.6)


        # Add the LaTeX object to the scene
        
        self.add_fixed_in_frame_mobjects(latex_equation)
        # Define Surface using the default values of PDF_bivariate_normal()
        # which represent the standard bivariate normal distribution
        distribution = always_redraw(
            lambda: Surface(
                lambda u, v: ax.c2p(
                    u, v, PDF_bivariate_normal(
                        u, v, mu_1=mu_1.get_value(), mu_2=mu_2.get_value(),
                        sigma_1 = sigma_1.get_value(), sigma_2 = sigma_2.get_value(),
                        rho = rho.get_value()
                    )
                ),
                resolution=(42, 42),
                u_range=[-4.5, 4.5],
                v_range=[-4.5, 4.5],
                fill_opacity=0.7
            ).set_fill_by_value(
                axes = ax,
                # Utilize color ramp colors with, higher values are "warmer"
                colors = [(COLOR_RAMP[0], 0),
                          (COLOR_RAMP[1], 0.05),
                          (COLOR_RAMP[2], 0.1),
                          (COLOR_RAMP[3], 0.15),
                          (COLOR_RAMP[4], 0.2)]
            )
        )
        

        # Set up animation
        self.add(ax, axis_labels,text,latex_equation)
        self.set_camera_orientation(
            phi=75*DEGREES,
            theta=-70*DEGREES,
            frame_center=[0, 0, 2],
            zoom=0.65)
        self.play(Create(distribution))
        self.move_camera(theta=70*DEGREES, run_time=2)
        self.wait(1.5)
        self.move_camera(theta=-70*DEGREES, run_time=2)
        self.wait()
        self.play(
            mu_1.animate.set_value(0), 
            sigma_1.animate.set_value(1), 
            mu_2.animate.set_value(0), 
            sigma_2.animate.set_value(1), 
            rho.animate.set_value(0), 
            run_time=1,
            rate_func=rate_functions.smooth
        )
        ##adjusting mu
        describe_mu = Tex(r'Adjusting means',font_size=35).next_to(text,DOWN,).shift(RIGHT*3.5)
        self.add_fixed_in_frame_mobjects(describe_mu)
        self.play(Write(describe_mu))
        self.wait()
        
        self.play(
            mu_1.animate.set_value(2), run_time=2,
            rate_func=rate_functions.smooth
        )
        self.wait()
        self.play(
            mu_1.animate.set_value(0), run_time=1,
            rate_func=rate_functions.smooth
        )
        self.play(
            mu_2.animate.set_value(-2), run_time=2,
            rate_func=rate_functions.smooth
        )
        self.wait()
        self.play(
            mu_2.animate.set_value(0), run_time=1,
            rate_func=rate_functions.smooth
        )
        # Top-down view
        self.move_camera(
            theta=-90*DEGREES,
            phi=0,
            frame_center=[0, 0, 0],
            zoom=0.5
        )
        self.play(
            mu_1.animate.set_value(-2),
            mu_2.animate.set_value(2),
            run_time=2,
            rate_func=rate_functions.smooth
        )
        self.wait()
        self.play(
            mu_1.animate.set_value(0),
            mu_2.animate.set_value(0),
            run_time=1,
            rate_fun=rate_functions.smooth
        )
        # Return camera to original position
        self.move_camera(
            theta=-70*DEGREES,
            phi=70*DEGREES,
            frame_center=[0, 0, 2],
            zoom=0.6
        )
        self.wait()
        self.remove(describe_mu)
        ##adjusting variance
        describe_var = Tex(r'Adjusting variance',font_size=35).next_to(text,DOWN,).shift(RIGHT*3.5)
        self.add_fixed_in_frame_mobjects(describe_var)
        self.play(Write(describe_var))
        self.play(
            sigma_1.animate.set_value(0.5),
            run_time=2,
            rate_func=rate_functions.smooth
        )
        self.wait()
        self.move_camera(theta=70*DEGREES, run_time=2)
        self.move_camera(theta=-70*DEGREES, run_time=2)
        self.play(
            sigma_1.animate.set_value(1),
            run_time=1,
            rate_func=rate_functions.smooth
        )
        self.wait()
        self.play(
            sigma_2.animate.set_value(1.5),
            run_time=2,
            rate_func=rate_functions.smooth
        )
        self.wait()
        self.move_camera(theta=70*DEGREES, run_time=2)
        self.move_camera(theta=-70*DEGREES, run_time=2)
        self.play(
            sigma_2.animate.set_value(1),
            run_time=1,
            rate_func=rate_functions.smooth
        )
        self.play(
            sigma_1.animate.set_value(1.5),
            sigma_2.animate.set_value(0.5),
            run_time=2,
            rate_func=rate_functions.smooth
        )
        self.wait()
        # Top-down view
        self.move_camera(
            theta=-90*DEGREES,
            phi=0,
            frame_center=[0, 0, 0],
            zoom=0.5
        )
        self.wait()
        self.play(
            sigma_1.animate.set_value(0.5),
            sigma_2.animate.set_value(1.5),
            run_time=1,
            rate_fun=rate_functions.smooth
        )
        self.wait(2)
        self.play(
            sigma_1.animate.set_value(0.75),
            sigma_2.animate.set_value(0.75),
        )
        # Return camera to starting position
        self.move_camera(
            theta=-70*DEGREES,
            phi=70*DEGREES,
            frame_center=[0, 0, 3],
            zoom=0.6
        )
        self.wait()
        self.play(
            sigma_1.animate.set_value(1),
            sigma_2.animate.set_value(1),
        )
        self.wait()
        self.remove(describe_var)
        ##Adjusting rho
        describe_corr = Tex(r'Adjusting correlation',font_size=35).next_to(text,DOWN,).shift(RIGHT*3.5)
        self.add_fixed_in_frame_mobjects(describe_corr)
        self.play(Write(describe_corr))
        # Animate to positive rho (positive correlation)
        self.play(
            rho.animate.set_value(0.75), run_time=2,
            rate_func=rate_functions.smooth
        )
        self.wait()
        # Look down on surface
        self.move_camera(
            theta=-90*DEGREES,
            phi=0,
            frame_center=[0, 0, 0],
            zoom=0.5
        )
        self.wait(2)
        # Return to starting camera position
        self.move_camera(
            theta=-70*DEGREES,
            phi=70*DEGREES,
            frame_center=[0, 0, 2.5],
            zoom=0.6
        )
        # Animate to negative rho (negative correlation)
        self.play(
            rho.animate.set_value(-0.75), run_time=2,
            rate_func=rate_functions.smooth
        )
        self.wait()
        # Look down on surface
        self.move_camera(
            theta=-90*DEGREES,
            phi=0,
            frame_center=[0, 0, 0],
            zoom=0.5
        )
        self.wait(2)
        # Animate to zero rho (no correlation)
        self.play(
            rho.animate.set_value(0), run_time=2,
            rate_func=rate_functions.smooth
        )
        # Return camera to starting position
        self.move_camera(
            theta=-70*DEGREES,
            phi=70*DEGREES,
            frame_center=[0, 0, 2.5],
            zoom=0.6
        )
        self.play(Uncreate(distribution))
        self.wait()
